In [ ]:
df['date'] = df[['観測時刻(年)','観測時刻(月)','観測時刻(日)']].apply(
    lambda row: '-'.join(row.map(lambda x: str(x).zfill(2))), axis=1
)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [ ]:
df['heat_ex'] = df.maxtemp.astype(float)>=35
df['heat'] = df.maxtemp.astype(float)>=30

df['heat_ex_prev1'] = df.maxtemp_prev1.astype(float)>=35
df['heat_prev1'] = df.maxtemp_prev1.astype(float)>=30

df['heat_ex_prev2'] = df.maxtemp_prev2.astype(float)>=35
df['heat_prev2'] = df.maxtemp_prev2.astype(float)>=30

In [ ]:
df.groupby('date')[['heat_ex','heat','heat_ex_prev1','heat_prev1','heat_ex_prev2','heat_prev2']].sum()

In [ ]:
#最新データの取得
#日付が変わると対象日も変わる
url = 'https://www.data.jma.go.jp/stats/data/mdrr/tem_rct/alltable/mxtemsadext00_rct.csv'
data = pd.read_csv(url, encoding='Shift-JIS')

In [ ]:
#前年を取得

url = 'https://www.data.jma.go.jp/obd/stats/etrn/view/temp/2022_hp.csv'
heatpoint_2022 = pd.read_csv(url, encoding='shift_jis')

In [ ]:
#column名をきれいにする
rename_dict = {col: '真夏日' if '真夏日' in col else ('猛暑日' if '猛暑日' in col else col) for col in heatpoint_2022.columns}
rename_dict.update({'年月日':'date'})
heatpoint_2022 = heatpoint_2022.rename(columns=rename_dict)
#日付変換
heatpoint_2022.date = pd.to_datetime(heatpoint_2022.date)

In [159]:
#気象庁
#過去7日間の最高気温のデータを取得するコード
#https://www.data.jma.go.jp/stats/data/mdrr/docs/csv_dl_readme.html
#ウェブ上のデータは毎日5:00ごろ更新＋修正は13:00、19:00、翌日1:00ごろの更新で反映

import pandas as pd
import re

#データを取得してファイル名に日付を入れて保存する
def get_previous_data(d):
    url = f'https://www.data.jma.go.jp/obd/stats/data/mdrr/tem_rct/alltable/mxtemsadext0{d}.csv'
    data = pd.read_csv(url, encoding='Shift-JIS')

    #日付
    yyyymmdd = f"{data['観測時刻(年)'][0]}{data.loc[0,['観測時刻(月)','観測時刻(日)']].apply(lambda x: str(x).zfill(2)).sum()}"
    #ファイル名
    
    filename = f"./data-maxtemp/daily-data/jma-maxtemp-{yyyymmdd}.csv"
    #dataにも記載
    data['date'] = pd.to_datetime(yyyymmdd, format='%Y%m%d')
    #csvで出力
    daily_data.to_csv(filename, index=False)
    return data

#実行
#集計（真夏・猛暑日を数える）
data_agg = pd.DataFrame()

###データ取得
for d in range(1, 8): 
    data = get_previous_data(d)
    ##ととのえる
    ##最高気温のデータのカラムを抽出
    ##（1日前＝昨日の~、2日前=一昨日の~、3日以上前=xx日の最高気温(℃)
    rename_dic = {col:'maxtemp' for col in data.columns if '日の最高気温(℃)' in col}
    data = data.rename(columns=rename_dic)
    ##統合
    data_agg = pd.concat([data_agg, data])

###計算
data_agg['over15'] = data_agg.maxtemp >= 15
data_agg['over20'] = data_agg.maxtemp >= 20
data_agg['over25'] = data_agg.maxtemp >= 25
data_agg['over30'] = data_agg.maxtemp >= 30
data_agg['over35'] = data_agg.maxtemp >= 35
data_agg['over40'] = data_agg.maxtemp >= 40
data_agg['over45'] = data_agg.maxtemp >= 45
data_agg['over50'] = data_agg.maxtemp >= 50
data_agg['total'] = 1
data_agg['null_values'] = data_agg.maxtemp.isna()

#集計df
heat_points_7days = data_agg.groupby(['date'])[['over15','over20','over25','over30','over35','over40','over45','over50','total','null_values']].sum().reset_index()

#過去分と統合
filename = "data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-ts.csv"
url = f"https://raw.githubusercontent.com/Nikkei-Visual-Data-Journalism/Heatwave/main/{filename}"
##過去分を取得
heat_points = pd.read_csv(url)
heat_points = pd.concat([heat_points,heat_points_7days])
##重複削除（新しく入ったほうを残す）
heat_points.date = pd.to_datetime(heat_points.date)
heat_points = heat_points.loc[~heat_points.duplicated(subset=['date'], keep='last')].reset_index(drop=True)
##出力
heat_points.to_csv(f"./{filename}", index=False)